# Gene2Vec

Gene2Vec is a method to learn gene embeddings from gene expression data. The method is based on the Word2Vec model, which is a popular method to learn word embeddings from text data. The idea is to learn a low-dimensional representation of genes that captures the relationships between genes based on their expression profiles. The gene embeddings can then be used for various downstream tasks, such as gene function prediction, gene expression analysis, and gene network analysis.

Gene2Vec was trained to produce 200-dimensional vectors for all human genes, using co-expression data obtained from 984 datasets. We can load it below:

In [ ]:
gene2vec_url = 'https://raw.githubusercontent.com/jingcheng-du/Gene2vec/master/pre_trained_emb/gene2vec_dim_200_iter_9_w2v.txt'

# Loading Gene2Vec

We can load the Gene2Vec model using the Gensim library. Gensim is a popular library for working with word embeddings, and it provides an easy-to-use interface for loading and working with pre-trained word embeddings. Let's load the Gene2Vec model and explore the embeddings:

In [ ]:
!pip install -U -q gensim

In [ ]:
from gensim.models import KeyedVectors

gene2vec = KeyedVectors.load_word2vec_format(gene2vec_url)

Now the word embeddings are loaded inside the `gene2vec` object. We can use this object to access the embeddings for individual genes and perform various operations, such as finding similar genes or performing vector arithmetic on gene embeddings. Let's explore some of these operations:

Load the embeddings for a specific gene:

In [ ]:
gene2vec['TP53']

Find the most similar genes to a given gene:

In [ ]:
gene2vec.most_similar('TP53')

Find the most similar genes to a combination of genes - this is a bit more complex, but you can think of this operation as finding genes that are similar to the combination of the two genes and different from the third gene:

In [ ]:
gene2vec.most_similar(positive=['TP53', 'BRCA1'], negative=['BRCA2'])

We can also apply dimensionality reduction to visualize the genes. For this, we'll use the Plotly library, because it'll allow us to mouse over the points and see the gene names. Let's visualize the gene embeddings using UMAP:

In [ ]:
!pip install -U -q umap-learn plotly

In [ ]:
import umap

embedding = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine').fit_transform(gene2vec.vectors)

In [ ]:
import plotly.express as px

fig = px.scatter(x=embedding[:, 0], y=embedding[:, 1], text=gene2vec.index_to_key)
fig.update_traces(textposition='top center')
fig.update_layout(title='Gene2Vec Embeddings')
fig.show()

# Augmenting data using embeddings

We can take advantage of these embeddings to augment performance in an existing task. Let's see how introducing these embeddings can improve our results on the HCC data:

In [ ]:
import pandas as pd
hcc = pd.read_csv('https://github.com/alexwolson/carte_workshop_2024/raw/main/data/HCC_all_ML_classification_test_annotated_frags_all_features_combined_4_tumors.csv.gz', compression='gzip')

In [ ]:
vocab = set(gene2vec.index_to_key)

In [ ]:
hcc = hcc[hcc.gene.isin(vocab)]

In [ ]:
hcc = hcc.sample(2000)

In [ ]:
categorical_columns = ['chr','upstream_motif','downstream_motif','Corrected_Call']
numerical_columns = ['frag','VAF','pos','read_cov','detected_read_cov','plasma_VAF','Corrected_Copy_Number']
y_column = 'alt_match'

X = hcc[categorical_columns + numerical_columns]
y = hcc[y_column]

In [ ]:
X = pd.get_dummies(X, columns=categorical_columns)

In [ ]:
X = X.values

In [ ]:
import numpy as np

gene_embeddings = hcc.gene.map(gene2vec.get_vector).values
gene_embeddings = np.stack(gene_embeddings)

print(gene_embeddings.shape)

In [ ]:
X = np.concatenate([X, gene_embeddings], axis=1)

In [ ]:
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a model

Now, train a model on the augmented data. You can import any model you'd like, such as a Random Forest or Gradient Boosting Machine.